<a href="https://colab.research.google.com/github/llawlaw23/Recommendation-System/blob/main/analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dask.dataframe as dd
import os
import gc
import re

In [ ]:
os.makedirs("cleaned", exist_ok = True)

In [ ]:
dtype_spec = {"property": "object"}

In [ ]:
props1 = pd.read_csv("Datasets/item_properties_part1.1.csv")
props2 = pd.read_csv("Datasets/item_properties_part2.csv")
props = pd.concat([props1, props2])

NameError: name 'dtype_spec' is not defined

In [ ]:
props.head()

In [ ]:
events = dd.read_csv("Datasets/events.csv")
cats = dd.read_csv("Datasets/category_tree.csv")

### Data Understanding & Preprocessing

#### 1. Data Overview
- Brief description of the dataset ( size, number of features, number of columns and rows, and observations)
- Types of variables (numerical, categorical, datetime, etc.)
- Initial observations about the data (e.g., imbalance, missing data, outliers)

#### 2. Data Quality Checks
- Check for missing values and filling or dropping them.
- dropping columns.
- checking for outliers.

In [ ]:
events["timestamp"] = dd.to_datetime(events["timestamp"], unit = "ms")
props["timestamp"] = dd.to_datetime(props["timestamp"], unit = "ms")

In [ ]:
num_rows = events.shape[0].compute()
num_cols = events.shape[1]

print(num_rows, num_cols)

In [ ]:
events.isnull().sum().compute()

In [ ]:
events = events.drop(columns = ["transactionid"])

In [ ]:
events.head()

In [ ]:
visitor_event = events.groupby("visitorid")["event"].count()

In [ ]:
Q1 = visitor_event.quantile(0.25)
Q3 = visitor_event.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

In [ ]:
outlier_visitor_ids = visitor_event[(visitor_event < lower_bound) | (visitor_event > upper_bound)].index.compute()

In [ ]:
is_outlier = events["visitorid"].isin(outlier_visitor_ids)

In [ ]:
events_F = events[~is_outlier]

In [ ]:
print(events_F.head())

In [ ]:
props.head()

In [ ]:
def sum_n_values(text):
    nums = re.findall(r"n([\d\.]+)", str(text))
    nums = [float(x) for x in nums]
    return sum(nums) if nums else None

props["value_n"] = props["value"].apply(sum_n_values, meta=("n_sum", "float64"))

In [ ]:
props = props.drop(columns = ["value"])

In [ ]:
props.head()

In [ ]:
props = props.dropna()

In [ ]:
props.head()

In [ ]:
cats.isnull().sum().compute()

In [ ]:
cats = cats.dropna()

In [ ]:
props = props.repartition(npartitions = 1)
props.to_parquet("cleaned/props.parquet", engine = "pyarrow", compression = "snappy")
del props; gc.collect()

In [ ]:
events_F = events_F.repartition(npartitions = 1)
events_F.to_parquet("cleaned/events.parquet", engine = "pyarrow", compression = "snappy")
del events_F; gc.collect()

In [ ]:
cats = cats.repartition(npartitions = 1)
cats.to_parquet("cleaned/cats.parquet", engine = "pyarrow", compression = "snappy")
del cats; gc.collect()

In [ ]:
# props = dd.read_parquet("props.parquet", engine="pyarrow")

# events_F = dd.read_parquet("events.parquet", engine="pyarrow")

cats = dd.read_parquet("cats.parquet", engine="pyarrow")

FileNotFoundError: An error occurred while calling the read_parquet method registered to the pandas backend.
Original Message: /content/cats.parquet

Creating My user-item matric and item feature Matrix

In [ ]:
from scipy.sparse import csr_matrix
import pandas as pd
# from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix
import numpy as np

In [ ]:
events_F.head()

,timestamp,visitorid,event,itemid,interaction_strength
0,2015-06-02 05:02:12.117,257597.0,view,355908.0,1
2,2015-06-02 05:13:19.827,111016.0,view,318965.0,1
3,2015-06-02 05:12:35.914,483717.0,view,253185.0,1
4,2015-06-02 05:02:17.106,951259.0,view,367447.0,1
5,2015-06-02 05:48:06.234,972639.0,view,22556.0,1


In [ ]:
event_weights = {"view": 1, "addtocart": 3, "transaction": 5}

events_F["interaction_strength"] = events_F["event"].map(
    event_weights,
    meta = ("interaction_strength", "int64")
)

In [ ]:
print("The 'interaction_strength' column has been created.")
print(events_F.head())

The 'interaction_strength' column has been created.
                timestamp  visitorid event    itemid  interaction_strength
0 2015-06-02 05:02:12.117   257597.0  view  355908.0                     1
2 2015-06-02 05:13:19.827   111016.0  view  318965.0                     1
3 2015-06-02 05:12:35.914   483717.0  view  253185.0                     1
4 2015-06-02 05:02:17.106   951259.0  view  367447.0                     1
5 2015-06-02 05:48:06.234   972639.0  view   22556.0                     1


In [ ]:
visitor_ids = events_F["visitorid"].unique()
item_ids = events_F["itemid"].unique()

In [ ]:
visitor_map = {visitor_id: i for i, visitor_id in enumerate(visitor_ids)}
item_map = {item_id: i for i, item_id in enumerate(item_ids)}

In [ ]:
events_F["visitor_idx"] = events_F["visitorid"].map(
    visitor_map,
     meta = ("visitor_idx", "int64")
)
events_F["item_idx"] = events_F["itemid"].map(
    item_map,
    meta = ("item_idx", "int64")
)

In [ ]:
events_computed_agg = events_F.groupby(["visitor_idx", "item_idx"])["interaction_strength"].sum().reset_index()

In [ ]:
events_computed = events_computed_agg.compute()

In [ ]:
rows = events_computed["visitor_idx"].values
cols = events_computed["item_idx"].values
data = events_computed["interaction_strength"].values

In [ ]:
user_item_matrix = csr_matrix((data, (rows, cols)), shape=(len(visitor_map), len(item_map)))

In [ ]:
print(f"Shape of the user-item interaction matrix: {user_item_matrix.shape}")

Shape of the user-item interaction matrix: (1287164, 210443)


In [ ]:
categories = props["property"].unique().compute().tolist()

In [ ]:
props["property"] = props["property"].astype(pd.CategoricalDtype(categories = categories))

In [ ]:
item_feature_matrix = props.pivot_table(
    index = "itemid",
    columns = "property",
    values = "value_n",
    aggfunc = "first"
)

In [ ]:
print(item_feature_matrix)

Dask DataFrame Structure:
                   888      400      790      451        0      566      134       19      881      188      663      619       60      917      202      719      964      681      810      175      631     1075      720      689      314      784      807      671      931      283      891      726      607      544      942      758     1083      225      987      204      569     1058     1066      141      884      761      674     1054      389      104      441      452      221      976      456      423      281      468      470      319      749      994      900      603      805      981      102      307      909      669      771      465      586      546      275      495      638      685      206      768      372      120       96      936       69      841      594      611      562      961      966      654      373      542       42      419      795       30      734      953      645      277      803       98      140      220       

Merging With Item properties and categories

In [ ]:
coo = user_item_matrix.tocoo()

In [ ]:
visitor_ids = events_F["visitorid"].unique().compute()
item_ids = events_F["itemid"].unique().compute()

In [ ]:
visitor2idx = {v: i for i, v in enumerate(visitor_ids)}
item2idx = {i: j for j, i in enumerate(item_ids)}

In [ ]:
events_idx = events_F.compute()
events_idx["visitor_idx"] = events_idx["visitor_idx"].astype("int32")
events_idx["item_idx"] = events_idx["item_idx"].astype("int32")

In [ ]:
events_idx = events_idx.dropna(subset=["visitor_idx", "item_idx"])

In [ ]:
coo = coo_matrix(
    (events_idx["interaction_strength"],
     (events_idx["visitor_idx"], events_idx["item_idx"]))
)

In [ ]:
df = pd.DataFrame({
    "visitor_idx": coo.row,
    "item_idx": coo.col,
    "interaction_strength": coo.data
})

In [ ]:
events_idx["visitor_idx"] = events_idx["visitorid"].map(visitor2idx)
events_idx["item_idx"] = events_idx["itemid"].map(item2idx)

In [ ]:
item_feature_matrix = item_feature_matrix.compute()

user_item_features = df.merge(
    item_feature_matrix,
    on="itemid",
    how="left"
)

MemoryError: Unable to allocate 1.87 GiB for an array with shape (251065907,) and data type int64

model fitting

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_df, test_df = train_test_split(
    user_item_features,
    test_size=0.2,
    random_state=42
)

In [ ]:

X_train = train_df.drop(["visitorid", "itemid", "interaction_strength"], axis=1)
y_train = train_df["interaction_strength"]

In [ ]:
X_test = test_df.drop(["visitorid", "itemid", "interaction_strength"], axis=1)
y_test = test_df["interaction_strength"]

In [ ]:
als_model = AlternatingLeastSquares(factors = 50, regularization = 0.1, iterations = 20)
als_model.fit(user_item_matrix)

In [ ]:
user_recs = {}
N = 10

for user_idx in range(user_item_matrix.shape[0]):
    recommended = als_model.recommend(user_idx, user_item_matrix[user_idx], N = N)
    user_recs[user_idx] = [(item_idx, score) for item_idx, score in recommended]

In [ ]:
recs_df = pd.DataFrame(
    [(user, item, score) for user, recs in user_recs.items() for item, score in recs],
    columns = ["visitor_idx", "item_idx", "score"]
)

#### Task 1 (Predict Add-to-Cart Properties)

In [ ]:
carts = events[events["event"] == "addtocart"]

### Task 2 (Detect Abnormal Users)

In [ ]:
unique_items = events.groupby("visitorid")["itemid"].nunique()

In [ ]:
event_counts = events.groupby("visitorid")["event"].count()

In [ ]:
user_stats = unique_items.to_frame("unique_items").merge(
    event_counts.to_frame("total_events"),
    left_index = True,
    right_index = True
)

In [ ]:
user_stats = user_stats.compute()

print(user_stats.head())

### Data Visualization and Analysing Business Question




- How do user demographics influence their preferences and interactions?

- What products are frequently purchased or viewed together?

- How do popularity metrics evolve monthly or seasonally?

- Which items frequently co-occur in transactions?

- How do preferences differ across age, location, or other demographics?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

Q1
- How many unique visitors per day and week?

In [ ]:
unique_visitors_daily = events.groupby(events["timestamp"].dt.date)["visitorid"].nunique().compute()

In [ ]:

print(unique_visitors_daily.head(10))

Q2
- What patterns exist in user-item interactions over time?